In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("../data/01_raw/titanic.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.shape

(891, 12)

In [6]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
spine_cols = ["PassengerId"]

In [9]:
spine = df[["PassengerId", "Survived"]].copy()

In [11]:
spine.head
spine

,PassengerId,Survived
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
886,887,0
887,888,1
888,889,0
889,890,1


### inzynieria cech

In [14]:
ftr_family_size = df[["PassengerId", "SibSp", "Parch"]].copy()
ftr_family_size["FamilySize"] = ftr_family_size["SibSp"] + ftr_family_size["Parch"]

tytul

In [15]:
threshold = 20
ftr_person_title = df[["PassengerId","Name"]].copy()
ftr_person_title["Title"] = ftr_person_title["Name"].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
title_names = (ftr_person_title['Title'].value_counts() < threshold)
ftr_person_title["Title"] = ftr_person_title["Title"].apply(lambda x: 'Others' if title_names.loc[x] == True else x)
ftr_person_title["Title"] = ftr_person_title["Title"].astype("category").cat.codes
ftr_person_title.drop(columns="Name", inplace=True)

In [16]:
ftr_others = df[["PassengerId", "Pclass", "Sex", "Age", "Fare", "Embarked"]].copy()
ftr_others["Age"].fillna(ftr_others["Age"].median(), inplace=True)
ftr_others["Sex"] = (ftr_others["Sex"] == "male").astype("int")
ftr_others["Embarked"] = ftr_others["Embarked"].astype("category").cat.codes

In [17]:
#laczenie kolumn
master = spine.copy()
for t in [ftr_family_size, ftr_person_title, ftr_others]:
    master = master.merge(t, on = spine_cols, how = "left")

In [18]:
master.columns

Index(['PassengerId', 'Survived', 'SibSp', 'Parch', 'FamilySize', 'Title',
       'Pclass', 'Sex', 'Age', 'Fare', 'Embarked'],
      dtype='object')

### Modelowanie 



In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
features = ['SibSp', 'Parch', 'FamilySize', 'Title',
       'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = "Survived"
param_dist = {
    "n_estimators": [10, 50, 100],
    "max_depth":[5, 6, 7, 8, 9, 10]
}

In [20]:
clf = RandomForestClassifier()
X = master[features]
y = master[target]

In [21]:
grid_search = GridSearchCV(
    clf, param_grid=param_dist, scoring="accuracy"
)
grid_search.fit(X, y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 50, 100]},
             scoring='accuracy')

### Ewaluacja

In [22]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [23]:
grid_search.best_params_

{'max_depth': 9, 'n_estimators': 100}

In [24]:
grid_search.best_score_

0.8327788588286988

In [25]:
# Access the best estimator and make predictions
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X)

# Calculate the evaluation metrics
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.936026936026936
Precision: 0.9672131147540983
Recall: 0.8625730994152047
F1-score: 0.9119010819165378
